In [2]:
import os
import json
import random
import numpy as np
import pandas as pd

from time import time
from tqdm import tqdm
from glob import glob

In [3]:
raw_data_path = './raw'
sample_data_path = './sample'
cdtx_path = 'zip_ncku_cdtx0001'
cust_f_path = 'zip_ncku_cust_f'

sample_chid_file = os.path.join(sample_data_path, 'sample_chid.txt')
cdtx_files = sorted(glob(os.path.join(raw_data_path, cdtx_path, '*.json')))
cust_f_files = sorted(glob(os.path.join(raw_data_path, cust_f_path, '*.json')))

sample_cdtx_file = os.path.join('sample_50k_cdtx.json')
sample_cust_f_file = os.path.join('sample_50k_cust_f.json')

In [4]:
# 寫檔案
def write_txt(file_name, lst):
    with open(file_name, 'w') as f:
        for i in tqdm(lst):
            f.write(f'{i}\n')

In [4]:
chid = set()

s_t = time()

for cust_f_file in tqdm(cust_f_files):
    with open(cust_f_file, "r") as f:
        cust_f = json.load(f)
    for index in cust_f.keys():
        chid.add(cust_f[index]['chid'])
        
print(f'cost time : {time()-s_t}')

  6%|▋         | 3/47 [01:33<22:58, 31.33s/it]


KeyboardInterrupt: 

In [81]:
write_txt('cdtx_chid.txt', chid)

100%|██████████| 2762314/2762314 [00:01<00:00, 1487552.76it/s]


In [5]:
cust_chid = set()
s_t = time()

dt = '2018-12-31'

for cust_f_file in tqdm(cust_f_files):
    with open(cust_f_file, "r") as f:
        cust_f = json.load(f)
    
    temp_dates = []
    for index in cust_f.keys():
        temp_dates.append(cust_f[index]['data_dt'].split('T')[0])
    
    if max(temp_dates) > dt:
        continue
    
    for index in cust_f.keys():
        cust_chid.add(cust_f[index]['chid'])
        
print(f'cost time : {time()-s_t}')

100%|██████████| 47/47 [18:27<00:00, 23.57s/it]

cost time : 1107.8042736053467


In [7]:
cdtx_chid = set()
s_t = time()

dt = '2018-12-31'

for cdtx_file in tqdm(cdtx_files):
    with open(cdtx_file, "r") as f:
        cdtx = json.load(f)
    
    temp_dates = []
    for index in cdtx.keys():
        temp_dates.append(cdtx[index]['csmdt'].split('T')[0])
    
    if max(temp_dates) > dt:
        continue
    
    for index in cdtx.keys():
        cdtx_chid.add(cdtx[index]['chid'])
        
print(f'cost time : {time()-s_t}')

100%|██████████| 151/151 [31:45<00:00, 12.62s/it]

cost time : 1905.9467680454254


In [8]:
inter_chid = chid & cdtx_chid

In [93]:
write_txt('chid_2018.txt', chid)

100%|██████████| 2274580/2274580 [00:01<00:00, 1459531.25it/s]


In [9]:
random.seed(1012+4028+4036)
sample_chids = random.sample(list(inter_chid), k = 50000)
print(len(sample_chids))

50000


In [10]:
write_txt('sample_50k_chid.txt', sample_chids)

100%|██████████| 50000/50000 [00:00<00:00, 1517805.60it/s]


In [11]:
sample_cdtx = {}
s_t = time()

count = -1

for cdtx_file in tqdm(cdtx_files):
    
    with open(cdtx_file, "r") as f:
        cdtx = json.load(f)
    temp_chids = []
    for index in cdtx.keys():
        chid = cdtx[index]['chid']
        temp_chids.append(chid)
        
    mask = np.isin(temp_chids, sample_chids)
    ids = np.where(mask == True)[0]
    for i in ids:
        count+=1
        sample_cdtx.update({str(count):cdtx[str(i)]})
            
print(f'cost time : {time()-s_t}')

100%|██████████| 151/151 [37:36<00:00, 14.94s/it]

cost time : 2256.29736828804


In [43]:
len(sample_cdtx)

5944391

In [12]:
with open(sample_cdtx_file, 'w') as f:
    json.dump(sample_cdtx, f)

# Sample cust_f

In [13]:
sample_cust_f = {}
s_t = time()

count = -1

for cust_f_file in tqdm(cust_f_files):
    
    with open(cust_f_file, "r") as f:
        cust_f = json.load(f)
    temp_chids = []
    for index in cust_f.keys():
        chid = cust_f[index]['chid']
        temp_chids.append(chid)
        
    mask = np.isin(temp_chids, sample_chids)
    ids = np.where(mask == True)[0]
    for i in ids:
        count+=1
        sample_cust_f.update({str(count):cust_f[str(i)]})
            
print(f'cost time : {time()-s_t}')

100%|██████████| 47/47 [21:50<00:00, 27.89s/it]

cost time : 1310.6392760276794


In [48]:
len(sample_chids)

50000

In [14]:
with open(sample_cust_f_file, 'w') as f:
    json.dump(sample_cust_f, f)

In [52]:
temp = []
for i in tqdm(sample_cdtx.items()):
    temp.append(i[1]['chid'])
    

100%|██████████| 5944391/5944391 [00:07<00:00, 844087.46it/s]


In [53]:
len(set(temp))

50000